In [7]:
import pytorch_lightning as pl
import torch
from torchvision import transforms
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader, random_split
from torchvision.models import resnet18, resnet34, resnet50
import torch.nn.functional as F
import torchmetrics

In [5]:
class CIFAR10DataModule(pl.LightningDataModule):
    def __init__(self, batch_size=32):
        super().__init__()
        self.batch_size = batch_size
        self.transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])

    def prepare_data(self):
        CIFAR10(root="data", train=True, download=True)
        CIFAR10(root="data", train=False, download=True)

    def setup(self, stage=None):
        if stage == 'fit' or stage is None:
            cifar_full = CIFAR10(root="data", train=True, transform=self.transform)
            self.cifar_train, self.cifar_val = random_split(cifar_full, [45000, 5000])
        if stage == 'test' or stage is None:
            self.cifar_test = CIFAR10(root="data", train=False, transform=self.transform)

    def train_dataloader(self):
        return DataLoader(self.cifar_train, batch_size=self.batch_size, shuffle=True)

    def val_dataloader(self):
        return DataLoader(self.cifar_val, batch_size=self.batch_size)

    def test_dataloader(self):
        return DataLoader(self.cifar_test, batch_size=self.batch_size)

In [17]:
class LitResNet(pl.LightningModule):
    def __init__(self, model_type):
        super().__init__()
        if model_type == 'resnet18':
            self.model = resnet18(pretrained=True)
        elif model_type == 'resnet34':
            self.model = resnet34(pretrained=True)
        elif model_type == 'resnet50':
            self.model = resnet50(pretrained=True)
        self.model.fc = torch.nn.Linear(self.model.fc.in_features, 10)
        
        self.acc = torchmetrics.Accuracy(task="multiclass", num_classes=10)
        self.precision = torchmetrics.Precision(task="multiclass", average='macro', num_classes=10)
        self.recall = torchmetrics.Recall(task="multiclass", average='macro', num_classes=10)
        self.f1 = torchmetrics.F1Score(task="multiclass", average='macro', num_classes=10)


    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        self.log('train_loss', loss)
        self.log('train_acc', self.acc(y_hat, y))
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        self.log('val_loss', loss)
        self.log('val_acc', self.acc(y_hat, y))
        self.log('precision', self.precision(y_hat, y))
        self.log('recall', self.recall(y_hat, y))
        self.log('f1_score', self.f1(y_hat, y))
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.model.parameters(), lr=1e-3)

In [19]:
data_module = CIFAR10DataModule()

for model_type in ['resnet18', 'resnet34', 'resnet50']:
    model = LitResNet(model_type)
    trainer = pl.Trainer(max_epochs=10)
    trainer.fit(model, data_module)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


100%|██████████████████████████████████████████████████████████████| 170498071/170498071 [00:15<00:00, 10720263.93it/s]


Extracting data\cifar-10-python.tar.gz to data
Files already downloaded and verified


Missing logger folder: C:\Users\dimit\Documents\notebooks\hw1\lightning_logs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type                | Params
--------------------------------------------------
0 | model     | ResNet              | 11.2 M
1 | acc       | MulticlassAccuracy  | 0     
2 | precision | MulticlassPrecision | 0     
3 | recall    | MulticlassRecall    | 0     
4 | f1        | MulticlassF1Score   | 0     
--------------------------------------------------
11.2 M    Trainable params
0         Non-trainable params
11.2 M    Total params
44.727    Total estimated model params size (MB)


Sanity Checking: |                                                                               | 0/? [00:00<…

...

Validation: |                                                                                    | 0/? [00:00<…

Validation: |                                                                                    | 0/? [00:00<…

Validation: |                                                                                    | 0/? [00:00<…

Validation: |                                                                                    | 0/? [00:00<…

`Trainer.fit` stopped: `max_epochs=10` reached.


In [20]:
# Загрузка расширения TensorBoard
%reload_ext tensorboard

# Запуск TensorBoard
%tensorboard --logdir=./lightning_logs